# Overview

We consider all graphs in Section $1.3$. For a given graph $\Gamma$, we determine whether the convex region $P_{ST}^{\sigma_{\Gamma}}$ is empty or not for all $\sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ (obtain via assignments.ipynb).

Note that if $P_{ST}^{\sigma_{\Gamma}}$ has dimension $|V(\Gamma)|-1$ (as it is a convex region-see definition) then $P_{ST}^{\sigma_{\Gamma}} \ne \emptyset$ and if $P_{ST}^{\sigma_{\Gamma}}$ has dimension strictly less than $|V(\Gamma)|-1$ then $P_{ST}^{\sigma_{\Gamma}} = \emptyset$.

In our methods, to show $P_{ST}^{\sigma_{\Gamma}} \ne \emptyset$ we modify the definition of $P_{ST}^{\sigma_{\Gamma}}$ with $<$ to $\le$, we do this in order to make use of the Sage function Polyhedron(). 

Mathematically we can still conclude that $P_{ST}^{\sigma_{\Gamma}} = \emptyset$ (with $<$) if $P_{ST}^{\sigma_{\Gamma}}$  (with $\le$) has dimension strictly less than $|V(\Gamma)|-1$. For each $\sigma_{\Gamma}$ we therefore determine the dimension $P_{ST}^{\sigma_{\Gamma}} \ne \emptyset$  (with $\le$) to conclude the existence/ non-existence of a numerical polarisation.

### Results

We see that all graphs consider in the Section $1.3$, with the expection of $F$ given by Filippo Viviani have $P_{ST}^{\sigma_{\Gamma}} \ne \emptyset$ for all $\sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$. For the graph $F$ there are $16$ stability conditions $\sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ such that $P_{ST}^{\sigma_{\Gamma}} = \emptyset$. It remains to show that these $16$ are in fact strong stability conditions, we do this in "6_strong_stability_check.ipynb".

### Notes


For a description of how we construct $P_{ST}^{\sigma_{\Gamma}}$ see Section "Functions/Construct $P_{ST}^{\sigma_{\Gamma}}$".

For each of the graphs in #Cases we load $\Sigma_{G,T,D_{T}}$ from "\Stability_Conditions\Graph\phi_investigation\examples\{graphname}".

We store the results as a list in "\Stability_Conditions\Graph\phi_investigation\examples\{graphname}\phi_analysis"

# Table of contents

1. [Functions](#s1)
    1. [Used throughout](#s11)
    2. [Construct $P_{ST}^{\sigma_{\Gamma}}$](#s12)
    3. [Loading and storing](#s13)
    4. [Analysis](#s14)
    5. [Describer functions](#s15)

2. [Cases](#s2)
    1. [Genus 1 $n$-necklace graphs $n \le 5$](#s21)
    2. [$\Gamma:=\Gamma_{1 1 k_3}$ for $2 \le k_3 \le 6$](#s22)
    3. [$\Gamma:=\Gamma_{1 k_2 k_3}$ for $k_2\ge 2$](#s23)
    4. [$\Gamma:=\Gamma_{k_1 k_2 k_3}$ for $k_1\ge 2$](#s24)
    5. [$\Gamma \in \{ G_1,G_2,G_3,F\}$](#s25)

# Functions <a name="s1"></a>

## Used throughout <a name="s11"></a>


In [ ]:
import numpy as np
import itertools
import pickle
from fractions import Fraction
from collections import Counter
import textwrap

## Construct $P_{ST}^{\sigma_{\Gamma}}$ <a name="s12"></a>

Given $\sigma_\Gamma \in \Sigma_{\Gamma}^{d}$ we follow the definition of $P_{ST}^{\sigma_{\Gamma}}$ given in Section $2.4$ of the thesis and give a method to construct $P_{ST}^{\sigma_{\Gamma}}$. Following this definition it is necessary to determine $P_{T}^{\sigma_{\Gamma}}$ for all $T \in \mathcal{ST}(\Gamma)$.

The main function of this section is get_intersection_tree_poly(), which gives the region $P_{ST}^{\sigma_{\Gamma}}$. We will now show how we arrive at this function:

1) First we record the information to necessary to state the set of inequalities of $P_{T}^{\sigma_{\Gamma}}$ for $T \in \mathcal{ST}(\Gamma)$ we do this with the function phi_tree_inequalities_data(). 

2) Next we construct the hyperplanes which give $P_{T}^{\sigma_{\Gamma}}$ for $T \in \mathcal{ST}(\Gamma)$ we do this with the function phi_tree_formatted_hyperplanes(). 

3) We then constuct $P_{T}^{\sigma_{\Gamma}}$ for each $T \in \mathcal{ST}(\Gamma)$ using get_tree_poly().

4) Then we take $P_{ST}^{\sigma_{\Gamma}}= \bigcap_{T \in \mathcal{ST}(\Gamma)}P_{T}^{\sigma_{\Gamma}}$ using get_intersection_tree_poly().

To help us understand the functions let us fix $\Gamma$ to be the $3$-necklace graph.

In [ ]:
# graphname="G3"
G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)
spanning_trees=[tree for tree in G.spanning_trees()]
T=spanning_trees[0]
# plot(G, figsize=3)

### Part 1: phi_tree_inequalities_data()

We now define the functions which will allow us to state the function phi_tree_inequalities_data which calculates the necessary components to produce $P_{T}^{\sigma_{\Gamma}}$.

We first need to determine the set of all elementary induced subgraphs $(\Gamma^{'},\Gamma^{'c})$.

In [16]:
def get_pairs_con_subg(G):# for phi_tree_inequalities_data
    
    """
    Note: we are not working with multigraphs so connected_subgraph_iterator should give subgraphs with all edges between    
    Objective: Used to get cut info and total degree adjustment
    Input:
    Return: Pairs of complete subgraphs for V and V^{c} such that V scup V^{c}=V(G)
    """
    
    vert_G=G.vertices()
    edges_G=G.edges()
    
    #get list of connected subgraphs:
    
    #Want to get pairs of complete subgraphs for pairs of vertices that is a is disjoint union of all verts
    ll=list(G.connected_subgraph_iterator())

    # put them into pairs, where vertices are a disjoint union of G.
    pairs=[]
    for i in ll:
        a=i.vertices()
        for j in ll:
            b=j.vertices()
            if sorted(a+b)==vert_G:
                #May need to take complete graph here containing a, and for b. 
                pairs.append((i,j))
                
#                 """
#                 We only need one subgraph in the cut, the following step prevents us having both subg in a cut.
#                 """
#                 if (i,j) in pairs:
#                     continue
#                 if (j,i) in pairs:
#                     continue
#                 else:
#                     pairs.append((i,j))
                    
#     for i in get_pairs_con_subg(G):
#         print(i[0].vertices(),i[1].vertices())
    return pairs

In [ ]:
#We see this gives all pair of elementary induced graphs
get_pairs_con_subg(G)

Let $\Gamma_0 :=T$. The next functions allow us to determine:

 the number of edges in a given cut $(\Gamma^{'},\Gamma^{'c})$ belonging to the tree and in particular

   $$\frac{\left|\operatorname{E}\left(\Gamma_0^c\right) \backslash (\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right))\right|}{2}$$
   
  the number of edges in a given cut $(\Gamma^{'},\Gamma^{'c})$ belonging to $\Gamma \setminus \Gamma_0$ and in particular
  
  $$\frac{\left|\operatorname{E}\left(\Gamma_0\right) \backslash\left(\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right)\right)\right|}{2}.$$


  

In [16]:
def get_cut_edges(G,pair): #for phi_tree_inequalities_data
    
    """
    Obj:Get list of the edges in a cut for a given pair, 
    for a pair of connected subgraphs that partitions the vertices of G.
    
    Return: list of edges in cut for a pair of connected subgraphs that partition the vertices of G 
    """
    a_edges=pair[0].edges()
    b_edges=pair[1].edges()
    total_edges=a_edges+b_edges
    
    edges_G=G.edges()

    #Cut edges.
    cut_edges=[ e for e in edges_G if e not in total_edges]
    
    return cut_edges

def edges_cut_inandnotin_tree(cut_edges,tree):#Used in phi_tree_inequalities_data
    
    """
    
    Obj: Consider cut_edges_G and ask if any of the edges of T are in cut_edges_G

    Returns: Number of edges in tree, and not in tree
    """

    tree_edges=tree.edges()

    e_in_tree=[edge for edge in cut_edges if edge in tree_edges]
    # print("edges of cut that belong to tree:",tree_edges_cut)

    e_notin_tree=[edge for edge in cut_edges if edge not in tree_edges]
    # print("edges of cut that do not belong to tree:",tree_comp_edges_cut)
    
    num_e_in_tree ,num_e_notin_tree=len(e_in_tree),len(e_notin_tree)
    
    return num_e_in_tree ,num_e_notin_tree

In [16]:
def get_gamma0_missing_edges(G,T,complete_subg):
    """
    Objective:find adjustment to degree of divisors, a number, to correct to the total degree.
    given by the number of edges in complete subgraph which are not in gamm0
    Input: gamma0 (the tree), and the complete_subg
    Return: number of edges complete_subg has over T (on one side of the  cut).
    
    
    Want |E(gamma0^c) cap E(gamma')| where gamma' is the complete subgraph and gamma0 the connected spanning subg.
    """
    
    
    gamma0_complement=[e for e in G.edges() if e not in T.edges()]
    
    gamma0_comp_inter_complete_subg=[e for e in gamma0_complement if e in complete_subg.edges()]
    
    return len(gamma0_comp_inter_complete_subg)


We now arrive at the first main function: phi_tree_inequalities_data which takes a tree T and returns a dictionary which records for each set of vertices of $\Gamma^{'}$ an elementary induced subgraphs $\Gamma^{'}$ the
 
 num_e_in_tree:
 the number of edges in the cut belonging to the tree
 
 num_e_notin_tree:
  the number of edges in the cut belonging to \Gamma \setminus tree

 gamma0_missing_edges:
  number of edges in $\Gamma^{'}$ which are not in $T$, that is $\left|\operatorname{E}\left(\Gamma_0^c\right) \cap \operatorname{E}\left(\Gamma^{\prime}\right)\right|$ for $\Gamma_0 =T$.
 

In [16]:
def phi_tree_inequalities_data(G,T): #Main
    
    """
    Objective: Used in tree_poly
    Inputs: 
    G: graph
    T:graph of spanning tree (or any spanning connected subgraph)
    
    Return:dictionary of data used to define the phi-tree inequalties. 
    Returns phi_G

    Dict contains the vertices of the cut : key
    List of [e_in_tree ,e_notin_tree]: value
    
    Where num_e_in_tree = edges of cut the belong to tree
         ,num_e_notin_tree= edges of cut that do not belong to tree
         
    terms=[num_e_in_tree ,num_e_notin_tree,gamma0_missing_edges]

    """
    
    #for main

    pairs=get_pairs_con_subg(G)

    inequ_data=[]
    for pair in pairs: 
        
        #As only care about one complete subg, we get the number of missing edges of T and complete_subg.
        complete_subg=pair[0]
        gamma0_missing_edges=get_gamma0_missing_edges(G,T,complete_subg)
        
        
        #Would like to know the edges that G has in the cut. i.e. cut_edges_G
        cut_edges=get_cut_edges(G,pair)

        num_e_in_tree ,num_e_notin_tree=edges_cut_inandnotin_tree(cut_edges,T)

        terms=[num_e_in_tree ,num_e_notin_tree,gamma0_missing_edges]
        
#         print("pair[0].vertices():",pair[0].vertices(),"pair[1].vertices():",pair[1].vertices())
#         print("[num_e_in_tree ,num_e_notin_tree]",terms)


        "Do we need to add the data for both subg in pair?"

        #get vertices of subg in pair and turn to "v1v2v3" format.
        subg1_l=pair[0].vertices()
        subg1 = ''.join(subg1_l)
        inequ_data.append((subg1,terms)) #subg1 is the indexor in phi_tree_formatted_hyperplanes

#         print("(subg1,terms):",(subg1,terms))
#         subg2_l=pair[1].vertices()
#         subg2 = ''.join(subg2_l)
#         inequ_data.append((subg2,terms))


    #Now join to phi-inequal data dict
    phi_tree_iequal_dict = {t[0]: t[1] for t in inequ_data}

    return phi_tree_iequal_dict

### Part 2: phi_tree_formatted_hyperplanes()

We now take the components of phi_tree_inequalities_data() and state phi_tree_formatted_hyperplanes(), which directly allows the construction of $P_{T}^{\sigma_{\Gamma}}$ for any $T \in \mathcal{ST}(\Gamma)$ using Polyhedron(). Before doing so we need the following functions to shorten the code.

For $\sigma_{\Gamma}^{d}$ we take $d=b_{1}(\Gamma)$ to be the first betti number of the graph.
As a consequence we have that for $\vec{d} \in \sigma_{\Gamma}(T)$ we have $deg(\vec{d})=0$ for any spanning tree $T$.

In [17]:
def get_first_betti_number(G):
    #for if the degree of divisors on trees is 0, the degree of lbm is this.
    return len(G.edges())-len(G.vertices())+1

For $\vec{d} \in \sigma_{\Gamma}(\Gamma_0)$ for any connected spanning tree or $\phi \in V^{d}(\Gamma)$, the following functions allows one to determine $d_{\Gamma^{'}}= \sum_{v \in V\left(\Gamma^{\prime}\right)}\vec{d}(v)$ and $\phi_{\Gamma^{'}}= \sum_{v \in V\left(\Gamma^{\prime}\right)}\phi(v)$.



In [17]:
def sum_comps(comp,indexor): #Done
    """returns the sum of the components of comp in the position by indexor"""
    #for d and phi
    sum_l=[comp[int(i)] for i in indexor] #Becuase of the vertex labeling of G having v_0....
    return sum(sum_l)

Recall for each  elementary induced subgraph $\Gamma^{'}$ defines two hyperplanes that give the bounds for $\phi_{\Gamma^{'}}$ that is is $L_{\Gamma_0,\Gamma^{'}}+ \vec{d}_{\Gamma^{'}}< \phi_{\Gamma^{'}}  <U_{\Gamma_0,\Gamma^{'}}+ \vec{d}_{\Gamma^{'}}$ (given by lower_part and upper_part).

Let $\vec{d} \in \sigma_{\Gamma}(\Gamma_0)$ and consider the pair of inequalities,

$$L_{\Gamma_0,\Gamma^{'}}< \phi_{\Gamma^{'}} - \vec{d}_{\Gamma^{'}} <U_{\Gamma_0,\Gamma^{'}}$$

The function lower_part() allows one to calculate 

$$\phi_{\Gamma^{'}} <U_{\Gamma_0,\Gamma^{'}} + \vec{d}_{\Gamma^{'}} .$$


and the function upper_part() allows one to calculate ($L_{\Gamma_0,\Gamma^{'}}+ \vec{d}_{\Gamma^{'}}< \phi_{\Gamma^{'}}$) but we write it in the following method so we can take the inequality in the same direction,

$$\phi_{\Gamma^{'}}<-L_{\Gamma_0,\Gamma^{'}}- \vec{d}_{\Gamma^{'}}$$



For the functions lower_part and upper_part the following terms correspond to:

1) T_term:
    
    $$\frac{\left|\operatorname{E}\left(\Gamma_0\right) \backslash\left(\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right)\right)\right|}{2}$$
    
2) T_comp_term:
    
    $$\frac{\left|\operatorname{E}\left(\Gamma_0^c\right) \backslash (\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right))\right|}{2}$$
    
3) divisor_comp_sum:
    
    $$d_{\Gamma^{'}}= \sum_{v \in V\left(\Gamma^{\prime}\right)}\vec{d}(v)$$
    
4) totaldeg_divisor_comp_sum:
    
    $$d_{\Gamma^{'}}+\left|\operatorname{E}\left(\Gamma_0^c\right) \cap \operatorname{E}\left(\Gamma^{\prime}\right)\right|$$
    
    

5) bterm:
    
    i) for lower_part is

    $$d_{\Gamma^{'}}+\left|\operatorname{E}\left(\Gamma_0^c\right) \cap \operatorname{E}\left(\Gamma^{\prime}\right)\right|+\frac{\left|\operatorname{E}\left(\Gamma_0\right) \backslash\left(\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right)\right)\right|}{2} +\frac{\left|\operatorname{E}\left(\Gamma_0^c\right) \backslash (\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right))\right|}{2}$$     

5) bterm:
    
    i) for upper_part is

    $$-d_{\Gamma^{'}}-\left|\operatorname{E}\left(\Gamma_0^c\right) \cap \operatorname{E}\left(\Gamma^{\prime}\right)\right|+\frac{\left|\operatorname{E}\left(\Gamma_0\right) \backslash\left(\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right)\right)\right|}{2} -\frac{\left|\operatorname{E}\left(\Gamma_0^c\right) \backslash (\operatorname{E}\left(\Gamma^{\prime}\right) \cup \operatorname{E}\left(\Gamma^{\prime c}\right))\right|}{2}$$     

In [17]:
def lower_part(indexor,n,divisor,num_e_in_tree,num_e_notin_tree,gamma0_missing_edges):

    #get b term
    T_term=num_e_in_tree*0.5
    
    T_comp_term=num_e_notin_tree*0.5
    
    divisor_comp_sum=sum_comps(divisor,indexor)
    
    totaldeg_divisor_comp_sum=divisor_comp_sum+gamma0_missing_edges#Corrected to total degree
    
    bterm=[totaldeg_divisor_comp_sum+T_term+T_comp_term] #See phi_tree_inequality_doc
    
    base=np.zeros(n)
    
    """
    It is very important whether its (1,i) or (-1,i) below in l
    """
    l=[(-1,i) for i in indexor] # put -1 in the ith position given be indexor. 

    for item in l:
        ipos = int(item[1]) # ith position
        val=item[0]
        base[ipos]=val

    xyz=base

    ineq=bterm+list(xyz) # [b,x,y,z...] want to concat                
    return ineq

def upper_part(indexor,n,divisor,num_e_in_tree,num_e_notin_tree,gamma0_missing_edges):

    #Remember upper part of modulus inequality.

    #get b term
    T_term=num_e_in_tree*0.5
    T_comp_term=num_e_notin_tree*0.5
    divisor_comp_sum=sum_comps(divisor,indexor)
    totaldeg_divisor_comp_sum=divisor_comp_sum+gamma0_missing_edges#Corrected to total degree

    bterm=[-totaldeg_divisor_comp_sum+T_term-T_comp_term] #See phi_tree_inequality_doc

    #get x,y,z term
    base=np.zeros(n)
    l=[(1,i) for i in indexor] # put -1 in the ith position given be indexor.

    for item in l:
        ipos = int(item[1]) # ith position
        val=item[0]
        base[ipos]=val

    xyz=base

    ineq=bterm+list(xyz) # [b,x,y,z...] want to concat

    return ineq

We can now define the function phi_tree_formatted_hyperplanes() which returns the set of hyperplanes used to define $P_{T}^{\sigma_{\Gamma}}$.

In [1]:
def phi_tree_formatted_hyperplanes(divisor,phi_G,G):
    
    """
    Objective: Get list of inquality data to feed into Sages Polyhedron function
    https://doc.sagemath.org/html/en/reference/discrete_geometry/sage/geometry/polyhedron/constructor.html
    
    Polyhedron(ieqs=[(0,1,0),(0,0,1),(1,-1,-1)]).Hrepresentation()
    (An inequality (-1, -1) x + 1 >= 0,
     An inequality (1, 0) x + 0 >= 0,
     An inequality (0, 1) x + 0 >= 0)
     
    to add the total degree inequality x+y+z..=degree.
    
    Ax=b
    
    Further Notes:    
    Here divisor is a single divisor. Not for all divisors in a set.
    
    Inputs:
    phi_G =  A dictionary that records indexs (vertices of \Gamma^{'}) of cuts (\Gamma^{'},\Gamma^{'c}) as keys, 
    and value=[num_e_in_tree,num_e_notin_tree]
    divisor= divisor in stability condition for tree ie. sigma_{\Gamma}(T)
    
    Returns: list of inequalities in the correct format to pass to Polyhedron as above.

    """
    
    #length of divisor:
    n=len(divisor)
    
    # We add the total degree equation to inequals for phi
    deg=get_first_betti_number(G) #will be 0 for trees.
    tot_equ1=[-deg,]+ [1]*n
    tot_equ2=[deg,]+ [-1]*n
    
    #List of inequalities we wish to return
    inequals=[tot_equ1,tot_equ2]     
        
    for indexor in list(phi_G.keys()):

        num_e_in_tree,num_e_notin_tree,gamma0_missing_edges=phi_G[indexor] #value=[num_e_in_tree,num_e_notin_tree]    

        res_upper=upper_part(indexor,n,divisor,num_e_in_tree,num_e_notin_tree,gamma0_missing_edges)

        res_lower=lower_part(indexor,n,divisor,num_e_in_tree,num_e_notin_tree,gamma0_missing_edges)

        res_combined= [res_lower,res_upper]

        inequals=inequals+res_combined
    return inequals

### Part 3: get_tree_poly()

The following function determine $P_{T}^{\sigma_{\Gamma}}$ using the information given by phi_tree_formatted_hyperplanes().

In [17]:
def get_tree_poly(G,T,divisor):#Main
    
    """
    Objective:Get P_{T}^{\sigma_{\Gamma}} for a tree using the phi-tree-inequalties.
    Input: 
    divisor: d given by \sigma_{\Gamma}(T)
    T: Spanning tree of \Gamma
    G is graph
    Returns: Polyhedron P_{T}^{\sigma_{\Gamma}} (with \le)
    """

    # get list of phi_tree_inequalities_data(G,T) for each trees.(dictionary)
    phi_G=phi_tree_inequalities_data(G,T)

    hyperp=phi_tree_formatted_hyperplanes(divisor,phi_G,G)

    #build a polytope.
    tree_poly=Polyhedron(ieqs=hyperp)

    return tree_poly

In [ ]:
#Tree_poly for G3 graph

#trivial tree assignment
assignments=[(spanning_trees[0],[0,0,0]),(spanning_trees[1],[0,0,0]),(spanning_trees[2],[0,0,0])]
tree_poly=get_tree_poly(G,T,assignments)

#Non-trivial tree assignment
assignments=[(spanning_trees[0],[0,0,0]),(spanning_trees[1],[0,1,-1]),(spanning_trees[2],[1,0,-1])]
tree_poly=get_tree_poly(G,T,assignments)

### Part 4: get_intersection_tree_poly()

The following function takes the intersection of $P_{T}^{\sigma_{\Gamma}}$ for all $T \in \mathcal{\Gamma}$  to give $P_{ST}^{\sigma_{\Gamma}}$.

In [17]:
def get_intersection_tree_poly(G,assignments):

    """
    Objective: get P_{ST}^{\sigma_{\Gamma}}
    Input: 
    assignments: the tree assignment A_{\Gamma} for \sigma_{\Gamma} as a list of pairs (Tree,divisor).
    Returns: P_{ST}^{\sigma_{\Gamma}} as a polyhedron (with \le)
    """
    
    #Build and record polytopes.
    poly_l=[]
    for ass in assignments:
    
        T,divisor=ass[0],ass[1]
    
        P=get_tree_poly(G,T,divisor)
        
        poly_l.append(P)

    #Take intersection of polytopes for trees
    inter=poly_l[0]
    for P in poly_l[1:]:
        inter=inter.intersection(P)

    return inter

For both unique stability conditions for the $3$-necklace graph we see that $P_{ST}^{\sigma_{\Gamma}}$ has dimension $2$.

In [3]:
#Trivial I3 example
assignments=[(spanning_trees[0],[0,0,0]),(spanning_trees[1],[0,0,0]),(spanning_trees[2],[0,0,0])]
inter_triv=get_intersection_tree_poly(G,assignments)
print(dim(inter_triv))

In [3]:
#Non-trivial I3 example
assignments=[(spanning_trees[0],[0,0,0]),(spanning_trees[1],[0,1,-1]),(spanning_trees[2],[1,0,-1])]
inter_nontriv=get_intersection_tree_poly(G,assignments)
print(dim(inter_nontriv))

## Loading and storing <a name="s13"></a>


The following functions load the pkl file stored in "examples/{graphname}/{graphname}.pkl" depending on the graphname choosen. Pickle_dic_l saves the output after we apply analyse_graphname() (see next section).

In [ ]:
def unpickle(graphname):
    # Unpickle the list
    filename=f"examples/{graphname}/{graphname}.pkl"
    with open(filename, 'rb') as f:
        unpickled_list = pickle.load(f)
    return unpickled_list

def pickle_dic_l(graphname,dic_l):
    # Pickle the list
    filename=f"examples/{graphname}/phi_analysis/dic_l.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(dic_l, f)
    return

# def unpickle_dic_l(graphname):
#     # Unpickle the list
#     filename=f"examples/{graphname}/phi_analysis/dic_l.pkl"
#     with open(filename, 'rb') as f:
#         unpickled_list = pickle.load(f)
#     return unpickled_list

The following function allows one to store a set of stability conditions and describe each as a txt file. 

In [ ]:
#Storing files 
def output_to_file(G,total_data,ordering_trees):#in text files
    """
    Objective: GEt text file descibing stability conditions
    Input: total_data is a set of stability conditions.
    ordering_trees is a list of spanning trees described by edges.
    Returns: text file
    
    Return to text file:
    - add list of subgraphs under trees.
    - record list of cycles for subg, after lbm. 
    """

    print(f"Information: Here are all stability conditions for the graph G")
    print(f"Graph: {G.edges()} \n")
    
    print(f"There are {len(total_data)} stability conditions for G. \n")
    
    print(f"The spanning trees of G are ordered as follows:")
    for index,tree in enumerate(ordering_trees):
        print(f"Tree {index+1}: {tree.edges()}" )
        
    print(f"\n The spanning subgraphs of G of the for T cup e are ordered as follows:")
    
    """
    Fix order of subg printed:
    
    Here get_subgraphs_gm1_Edgesmissing does not reflect the ordering of subg in subg_cycle_l=tup[3]
    subgraphs= get_subgraphs_gm1_Edgesmissing(G) #Get #\tilde{G} the subgraphs Tcup e.-D

    #Assuming subg_cycle_l for each element of total_data is ordered the same subg.
    """

    tup_example=total_data[0]
    subg_cycle_l=tup_example[3]
    subg_ll=[pair[0] for pair in subg_cycle_l]

    for index,subg in enumerate(subg_ll):
        print(f"Subgraph {index+1}: {subg.edges()}" )
    

    print("\n We will now list all stabilty conditions. The data will be presented in three lines:")
    print("   1. The assignment datum on trees the order of which will be given by the Tree order,")
    print("   2. The union of sets of the form assignment datum of T + the set of break divisors for T, " )          
    print("   3. The cycle used with th subgraph to obtain the assignments on trees (ordered by subgraphs above).\n")

    for tup in total_data:
        
        print(tup[0]) #assignment datum
        
        #Put sigma to list format
        print(tup[1].tolist()) #lbm
        
        subg_cycle_l=tup[3]
        l_cycles=[(x[1],x[2].edges()) for x in subg_cycle_l] #records cycle and tree for sub.
        print(l_cycles)
        
        print("\n")
    
    return

The following function are specific to the storing of a text description and pkl file of the output of the function analyse_graphname_FV()

In [ ]:
def pickle_subset_true(graphname,l):
    # Pickle the list
    filename=f"{graphname}_top_dim.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(l, f)
    f.close()  # Close the file after writing

    return

def pickle_subset_false(graphname,l):
    # Pickle the list
    filename=f"{graphname}_degen.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(l, f)
    f.close()  # Close the file after writing

    return

def store_data_top_dim(G,total_data,ordering_trees,graphname="test"):
    name=f"{graphname}_top_dim.txt"
    print(name)
    with open(name, "w") as f:
        sys.stdout = f
#         print("before output")
        output_to_file(G,total_data,ordering_trees)
#         print("after output")
        
        sys.stdout = sys.__stdout__
#         print("here closes??")
    f.close()  # Close the file after writing
#     print("does it close????")
    return

def store_data_degen(G,total_data,ordering_trees,graphname="test"):
    name=f"{graphname}_degen.txt"
    print(name)
    with open(name, "w") as f:
        sys.stdout = f

        output_to_file(G,total_data,ordering_trees)

        sys.stdout = sys.__stdout__
    f.close()  # Close the file after writing

    return

## Analysis <a name="s14"></a>

We now determine for each stability condition the dimension of $P_{ST}^{\sigma_{\Gamma}}$.
using get_intersection_tree_poly()

In [ ]:
def analyse_graphname(G,data,start=0,terminator=False,ind_breaks=0): #general not FV example
    """
    Obj: Measure for each stability condition the dimension of P_{ST}^{\sigma_{\Gamma}}
    #inputs:   
    
    size: number of vertices of graph minus 1.
    graphname:name in folder to load the data
    G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)
    #     start: the ith term 1,...,k you want to start the computation from.
    #     terminator = is a number 1,...,k, the nth to which we calculate upo to so 
    not to have to calc all cases (used for examples).
    
    Returns: a list of dictionaries of the form {"Dimension of tree-poly":dim_tree_poly}
    where tree-poly is the region P_{ST}^{\sigma_{\Gamma}} for each stability codnition
    and dim_tree_poly is its dimension.
    """
    
    dic_l=[]
    top_dim=[] # record those with top dim poly
    degen=[] #record wsc with degen poly.
    
    for term,wsc in enumerate(data):
        
        #Use start to skip to the case you want to study.
        if term<start-1:
            continue
        
        print("Start=",start,"----",term,"----terminator:",terminator)
        
        #for Tree_poly
        assignments=wsc[2] #[(Graph on 3 vertices, [0, 0, 0]),(Graph on 3 vertices, [0, 0, 0]),(Graph on 3 vertices, [0, 0, 0])]
        tree_poly=get_intersection_tree_poly(G,assignments)
        dic={"Dimension of tree-poly":dim(tree_poly)}
        dic_l.append(dic)

    
    return dic_l

As the graph $F$ is known to have cases where the dimension of $P_{ST}^{\sigma_{\Gamma}}$ is strictly less than $|V(\Gamma)|-1$ we record those cases.

In [ ]:
def analyse_graphname_FV(size,graphname,G,data,start=0,terminator=False): 
    """
    Obj: Measure for each stability condition the dimension of P_{ST}^{\sigma_{\Gamma}}
    As it has fail cases when dim_tree_poly< |V(\Gamma)|-1

    #inputs:    
    size: number of vertices of graph minus 1.
    graphname:name in folder to load the data
    G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)
    #     start: the ith term 1,...,k you want to start the computation from.
    #     terminator = is a number 1,...,k, the nth to which we calculate upo to so 
    not to have to calc all cases (used for examples).
    
    Returns: a list of dictionaries of the form {"Dimension of tree-poly":dim_tree_poly}
    where tree-poly is the region P_{ST}^{\sigma_{\Gamma}} for each stability codnition
    and dim_tree_poly is its dimension.

    """
    dic_l=[]
    top_dim=[] # record those with top dim poly
    degen=[] #record wsc with degen poly.
    
    for term,wsc in enumerate(data):
        
        #Use start to skip to the case you want to study.
        if term<start-1:
            continue
        print("Start=",start,"----",term,"----terminator:",terminator)
        
        #for Tree_poly
        assignments=wsc[2] #[(Graph on 3 vertices, [0, 0, 0]),(Graph on 3 vertices, [0, 0, 0]),(Graph on 3 vertices, [0, 0, 0])]
        tree_poly=get_intersection_tree_poly(G,assignments)
        dim_tree_poly=dim(tree_poly)
        
        print("dim_tree_poly:",dim_tree_poly)
        dic={"Dimension of tree-poly":dim_tree_poly}
        dic_l.append(dic)
        
        if dim_tree_poly==size:
            top_dim.append(wsc)
        else:
            print(f"{term}: WSC not phi: \n",wsc)
            degen.append(wsc)
            
    #Store those wsc that fail and those that are top dim poly.
    wsc_ex=data[0] #example wsc to get assignment to get trees in order.
    ass_ex=wsc_ex[2] # [(T,ass),...]
    ordering_trees=[pair[0] for pair in ass_ex]
    pickle_subset_true(graphname,top_dim)
    store_data_top_dim(G,top_dim,ordering_trees,graphname)
    pickle_subset_false(graphname,degen)
    store_data_degen(G,degen,ordering_trees,graphname)
    
    return dic_l

The following function is analogous to analyse_graphname() expect we record those cases where $P_{ST}^{\sigma_{\Gamma}}$ the dimension is less than $|V(\Gamma)|-1$, and those that have dimension $|V(\Gamma)|-1$.

In [ ]:
def analyse_graphname_FV(size,graphname,G,data,start=0,terminator=False): 
    """
    Obj: Create a table of results for the graph F: 
    As it has fail cases when dim_tree_poly< |V(\Gamma)|-1

    #inputs:    
    size: number of vertices of graph minus 1.
    graphname:name in folder to load the data
    G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)
    #     start: the ith term 1,...,k you want to start the computation from.
    #     terminator = is a number 1,...,k, the nth to which we calculate upo to so 
    not to have to calc all cases (used for examples).
    
    Returns: a list of dictionaries of the form {"Dimension of tree-poly":dim_tree_poly}
    where tree-poly is the region P_{ST}^{\sigma_{\Gamma}} for each stability codnition
    and dim_tree_poly is its dimension.

    """
    dic_l=[]
    top_dim=[] # record those with top dim poly
    degen=[] #record wsc with degen poly.
    
    for term,wsc in enumerate(data):
        
        #Use start to skip to the case you want to study.
        if term<start-1:
            continue
        print("Start=",start,"----",term,"----terminator:",terminator)
        
        #for Tree_poly
        assignments=wsc[2] #[(Graph on 3 vertices, [0, 0, 0]),(Graph on 3 vertices, [0, 0, 0]),(Graph on 3 vertices, [0, 0, 0])]
        tree_poly=get_intersection_tree_poly(G,assignments)
        dim_tree_poly=dim(tree_poly)
        
        print("dim_tree_poly:",dim_tree_poly)
        dic={"Dimension of tree-poly":dim_tree_poly}
        dic_l.append(dic)
        
        if dim_tree_poly==size:
            top_dim.append(wsc)
        else:
            print(f"{term}: WSC not phi: \n",wsc)
            degen.append(wsc)
            
    #Store those wsc that fail and those that are top dim poly.
    wsc_ex=data[0] #example wsc to get assignment to get trees in order.
    ass_ex=wsc_ex[2] # [(T,ass),...]
    ordering_trees=[pair[0] for pair in ass_ex]
    
    #We store the data
    pickle_subset_true(graphname,top_dim)
    store_data_top_dim(G,top_dim,ordering_trees,graphname)
    pickle_subset_false(graphname,degen)
    store_data_degen(G,degen,ordering_trees,graphname)
    
    return dic_l

The following function returns a summary the results of analyse_graphname() and analyse_graphname_FV().

In [ ]:
def analyse_table(dic_l):
    
    """
    Objective: Give a summary of results
    Input: list of dic={"Dimension of tree-poly":dim(tree_poly) }  
    Return:Summary text of results
    """
    
    num_wsc=[]#number of wsc
    dim_value_counts=[]

    for dic in dic_l:
    
        num_wsc.append(dic["Dimension of tree-poly"])
        dim_value_counts.append(dic["Dimension of tree-poly"])

    dim_value_counts_dict=dict(Counter(dim_value_counts)) # dictionary of number of terms of a given dim    
    print(textwrap.dedent(f'''\
        Total number of stability conditions {len(num_wsc)}
        Number of polytopes of a given (dimension:number of at dim): {dim_value_counts_dict}
    '''))
    return 

## Describer functions <a name="s15"></a>


The following functions help with documentation, please ignore.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import ast
import inspect

def find_functions_used(func):
    
    """Add print Objective."""
    
    functions_used = set()
    tree = ast.parse(inspect.getsource(func))
    for node in ast.walk(tree):
        if isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
            functions_used.add(node.func.id)
    return functions_used

def find_functions_used_l(l):
    for func in l:
        functions_used = find_functions_used(func)
        print(f"{func.__name__}:{functions_used} \n")

# l = [f1, f2, f3]
# find_functions_used_l(l)

# Cases <a name="s2"></a>

## Genus 1 $n$-necklace graphs $n \le 5$ <a name="s21"></a>

Even though we know by Section $3.1$ that every genus $1$ graphs every stability conditions is given by a numerical polatisation, we check the first few cases to ensure the methods are correct.

### $3$-necklace

In [29]:
graphname="G3"
G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)

data=unpickle(graphname) #Size=2

In [30]:
print(data)

[([[0, 0, 0], [0, 0, 0], [0, 0, 0]], array([[0, 0, 1],
       [0, 1, 0],
       [1, 0, 0]], dtype=int32), [(Graph on 3 vertices, [0, 0, 0]), (Graph on 3 vertices, [0, 0, 0]), (Graph on 3 vertices, [0, 0, 0])], [(Multi-graph on 3 vertices, (1, 3, 2), Multi-graph on 3 vertices)]), ([[0, 0, 0], [0, 1, -1], [1, 0, -1]], array([[ 0,  1,  0],
       [ 1,  0,  0],
       [ 1,  1, -1]], dtype=int32), [(Graph on 3 vertices, [0, 0, 0]), (Graph on 3 vertices, [0, 1, -1]), (Graph on 3 vertices, [1, 0, -1])], [(Multi-graph on 3 vertices, (1, 2, 3), Multi-graph on 3 vertices)])]


In [31]:
dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

Start= 0 ---- 0 ----terminator: False
Start= 0 ---- 1 ----terminator: False


In [32]:
analyse_table(dic_l,f=False)

Total number of wsc 2
Number of polytopes of a given (dimension:number of at dim): {2: 2}
Number of wsc that are/are not generated by phi_av: {True: 2}
Number of wsc that are/are not generated by phi_can: None



### $4$-necklace

In [78]:
graphname="G4"
G=Graph([("0","1"),("1","2"),("2","3"),("3","0")], multiedges=True)

data=unpickle(graphname) #Size=6

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

Start= 0 ---- 0 ----terminator: False
Start= 0 ---- 1 ----terminator: False
Start= 0 ---- 2 ----terminator: False
Start= 0 ---- 3 ----terminator: False
Start= 0 ---- 4 ----terminator: False
Start= 0 ---- 5 ----terminator: False


In [81]:
analyse_table(dic_l,f=False)

Total number of wsc 6
Number of wsc that do/do not satisfy graph poly == tree poly: {True: 6}
Number of polytopes of a given (dimension:number of at dim): {3: 6}
Number of wsc that are/are not generated by phi_rep: {True: 6}
Number of wsc that are/are not generated by phi_av: {True: 6}
Number of wsc that are/are not generated by phi_can: None
Number of wsc that are/are not generated by phi_av_translated1: None
Number of wsc that are/are not generated by phi_av_translated2: None



### $5$-necklace

In [82]:
graphname="G5"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0")], multiedges=True)

data=unpickle(graphname) #Size=24

dic_l=analyse_graphname(G,data)

#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

Start= 0 ---- 0 ----terminator: False
Start= 0 ---- 1 ----terminator: False
Start= 0 ---- 2 ----terminator: False
Start= 0 ---- 3 ----terminator: False
Start= 0 ---- 4 ----terminator: False
Start= 0 ---- 5 ----terminator: False
Start= 0 ---- 6 ----terminator: False
Start= 0 ---- 7 ----terminator: False
Start= 0 ---- 8 ----terminator: False
Start= 0 ---- 9 ----terminator: False
Start= 0 ---- 10 ----terminator: False
Start= 0 ---- 11 ----terminator: False
Start= 0 ---- 12 ----terminator: False
Start= 0 ---- 13 ----terminator: False
Start= 0 ---- 14 ----terminator: False
Start= 0 ---- 15 ----terminator: False
Start= 0 ---- 16 ----terminator: False
Start= 0 ---- 17 ----terminator: False
Start= 0 ---- 18 ----terminator: False
Start= 0 ---- 19 ----terminator: False
Start= 0 ---- 20 ----terminator: False
Start= 0 ---- 21 ----terminator: False
Start= 0 ---- 22 ----terminator: False
Start= 0 ---- 23 ----terminator: False


In [83]:
analyse_table(dic_l,f=False)

Total number of wsc 24
Number of wsc that do/do not satisfy graph poly == tree poly: {True: 24}
Number of polytopes of a given (dimension:number of at dim): {4: 24}
Number of wsc that are/are not generated by phi_rep: {True: 24}
Number of wsc that are/are not generated by phi_av: {True: 24}
Number of wsc that are/are not generated by phi_can: None
Number of wsc that are/are not generated by phi_av_translated1: None
Number of wsc that are/are not generated by phi_av_translated2: None



## $\Gamma:=\Gamma_{1 1 k_3}$ for $2 \le k_3 \le 6$  <a name="s22"></a>

We now consider the dimension of $P_{ST}^{\sigma_{\Gamma}}$ for $ \sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ for graphs  

$\Gamma:=\Gamma_{k_1 k_2 k_3}$ with $(k_1,k_2,k_3)=\{(1,1,2),(1,1,3),(1,1,4),(1,1,5),(1,1,6)\}$. 

### $(k_1,k_2,k_3)=(1,1,2)$

In [17]:
graphname="G3M1" 
G=Graph([("0","1","e11"),("1","2","e2"),("2","0","e3"),("0","1","e12")], multiedges=True)
# spanning_trees_e=[tree for tree in GM1e.spanning_trees(labels=True)]

data=unpickle(graphname) #Size=2

dic_l=analyse_graphname(G,data)

Start= 0 ---- 0 ----terminator: False
Start= 0 ---- 1 ----terminator: False


In [18]:
analyse_table(dic_l,f=True)

Total number of wsc 2
Number of wsc that do/do not satisfy graph poly == tree poly: {True: 2}
Number of polytopes of a given (dimension:number of at dim): {2: 2}
Number of wsc that are/are not generated by phi_rep: {True: 2}
Number of wsc that are/are not generated by phi_av: {True: 2}
Number of wsc that are/are not generated by phi_can: {True: 1, False: 1}
Number of wsc that are/are not generated by phi_av_translated1: {True: 2}
Number of wsc that are/are not generated by phi_av_translated2: {True: 2}



In [19]:
# pickle_dic_l(graphname,dic_l)

### $(k_1,k_2,k_3)=(1,1,3)$

In [22]:
graphname="G4M1" 
G=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","0","e4"),("0","1","e12")], multiedges=True)

data=unpickle(graphname) #Size=6

dic_l=analyse_graphname(G,data)

Start= 0 ---- 0 ----terminator: False
Start= 0 ---- 1 ----terminator: False
Start= 0 ---- 2 ----terminator: False
Start= 0 ---- 3 ----terminator: False
Start= 0 ---- 4 ----terminator: False
Start= 0 ---- 5 ----terminator: False


In [23]:
analyse_table(dic_l,f=True)

Total number of wsc 6
Number of wsc that do/do not satisfy graph poly == tree poly: {True: 6}
Number of polytopes of a given (dimension:number of at dim): {3: 6}
Number of wsc that are/are not generated by phi_rep: {True: 6}
Number of wsc that are/are not generated by phi_av: {True: 6}
Number of wsc that are/are not generated by phi_can: {True: 1, False: 5}
Number of wsc that are/are not generated by phi_av_translated1: {True: 6}
Number of wsc that are/are not generated by phi_av_translated2: {True: 6}



In [24]:
pickle_dic_l(graphname,dic_l)

### $(k_1,k_2,k_3)=(1,1,4)$

In [25]:
graphname="G5M1" 
G=Graph([("0","1","e11"),("1","2","e2"),("2","3","e3"),("3","4","e4"),("4","0","e5"),("0","1","e12")], multiedges=True)

data=unpickle(graphname) #Size=24

dic_l=analyse_graphname(G,data)

Start= 0 ---- 0 ----terminator: False
Start= 0 ---- 1 ----terminator: False
Start= 0 ---- 2 ----terminator: False
Start= 0 ---- 3 ----terminator: False
Start= 0 ---- 4 ----terminator: False
Start= 0 ---- 5 ----terminator: False
Start= 0 ---- 6 ----terminator: False
Start= 0 ---- 7 ----terminator: False
Start= 0 ---- 8 ----terminator: False
Start= 0 ---- 9 ----terminator: False
Start= 0 ---- 10 ----terminator: False
Start= 0 ---- 11 ----terminator: False
Start= 0 ---- 12 ----terminator: False
Start= 0 ---- 13 ----terminator: False
Start= 0 ---- 14 ----terminator: False
Start= 0 ---- 15 ----terminator: False
Start= 0 ---- 16 ----terminator: False
Start= 0 ---- 17 ----terminator: False
Start= 0 ---- 18 ----terminator: False
Start= 0 ---- 19 ----terminator: False
Start= 0 ---- 20 ----terminator: False
Start= 0 ---- 21 ----terminator: False
Start= 0 ---- 22 ----terminator: False
Start= 0 ---- 23 ----terminator: False


In [32]:
analyse_table(dic_l,f=True)

Total number of wsc 24
Number of wsc that do/do not satisfy graph poly == tree poly: {True: 24}
Number of polytopes of a given (dimension:number of at dim): {4: 24}
Number of wsc that are/are not generated by phi_rep: {True: 24}
Number of wsc that are/are not generated by phi_av: {True: 24}
Number of wsc that are/are not generated by phi_can: {True: 1, False: 23}
Number of wsc that are/are not generated by phi_av_translated1: {True: 24}
Number of wsc that are/are not generated by phi_av_translated2: {True: 24}



In [27]:
pickle_dic_l(graphname,dic_l)

### $(k_1,k_2,k_3)=(1,1,5)$

### $(k_1,k_2,k_3)=(1,1,6)$

## $\Gamma:=\Gamma_{1 k_2 k_3}$ for $k_2\ge 2$  <a name="s23"></a>

We now consider the dimension of $P_{ST}^{\sigma_{\Gamma}}$ for $ \sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ for graphs   $\Gamma:=\Gamma_{k_1 k_2 k_3}$ with $(k_1,k_2,k_3)=\{(1,2,2),(1,2,3),(1,2,4),(1,3,3),(1,2,5),(1,3,4)\}$. 

### $(k_1,k_2,k_3)=(1,2,2)$

In [43]:
graphname="G4M2" 
G=Graph([("0","1"),("1","2"),("2","3"),("3","0"),("0","2")], multiedges=True)

data=unpickle(graphname) #Size=6

dic_l=analyse_graphname(G,data)

Start= 0 ---- 0 ----terminator: False
Start= 0 ---- 1 ----terminator: False
Start= 0 ---- 2 ----terminator: False
Start= 0 ---- 3 ----terminator: False
Start= 0 ---- 4 ----terminator: False
Start= 0 ---- 5 ----terminator: False


In [70]:
#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [49]:
analyse_table(dic_l,f=True)

Total number of wsc 6
Number of polytopes of a given (dimension:number of at dim): {3: 6}
Number of wsc that are/are not generated by phi_av: {True: 6}
Number of wsc that are/are not generated by phi_can: {True: 1, False: 5}
Number of wsc that are/are not generated by phi_av_translated1: {False: 6}
Number of wsc that are/are not generated by phi_av_translated2: {False: 1, True: 5}
Number of wsc that are/are not generated by phi_av_translated3: {True: 5, False: 1}
Number of wsc that are/are not generated by phi_av_translated4: {True: 1, False: 5}
Number of wsc that are/are not generated by phi_av_translated5: {False: 6}
Number of wsc that are/are not generated by phi_av_translated6: {False: 6}
Number of wsc that are/are not generated by phi_av_translated7: {False: 6}
Number of wsc that are/are not generated by phi_av_translated8: {False: 6}        




### $(k_1,k_2,k_3)=(1,2,3)$

In [86]:
graphname="G5M2"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2")], multiedges=True)

data=unpickle(graphname) #Size=24

dic_l=analyse_graphname(G,data)

Start= 0 ---- 0 ----terminator: False
Start= 0 ---- 1 ----terminator: False
Start= 0 ---- 2 ----terminator: False
Start= 0 ---- 3 ----terminator: False
Start= 0 ---- 4 ----terminator: False
Start= 0 ---- 5 ----terminator: False
Start= 0 ---- 6 ----terminator: False
Start= 0 ---- 7 ----terminator: False
Start= 0 ---- 8 ----terminator: False
Start= 0 ---- 9 ----terminator: False
Start= 0 ---- 10 ----terminator: False
Start= 0 ---- 11 ----terminator: False
Start= 0 ---- 12 ----terminator: False
Start= 0 ---- 13 ----terminator: False
Start= 0 ---- 14 ----terminator: False
Start= 0 ---- 15 ----terminator: False
Start= 0 ---- 16 ----terminator: False
Start= 0 ---- 17 ----terminator: False
Start= 0 ---- 18 ----terminator: False
Start= 0 ---- 19 ----terminator: False
Start= 0 ---- 20 ----terminator: False
Start= 0 ---- 21 ----terminator: False
Start= 0 ---- 22 ----terminator: False
Start= 0 ---- 23 ----terminator: False


In [87]:
#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [88]:
analyse_table(dic_l,f=True)

Total number of wsc 24
Number of wsc that do/do not satisfy graph poly == tree poly: {True: 24}
Number of polytopes of a given (dimension:number of at dim): {4: 24}
Number of wsc that are/are not generated by phi_rep: {True: 24}
Number of wsc that are/are not generated by phi_av: {True: 24}
Number of wsc that are/are not generated by phi_can: {True: 1, False: 23}
Number of wsc that are/are not generated by phi_av_translated1: {True: 16, False: 8}
Number of wsc that are/are not generated by phi_av_translated2: {False: 8, True: 16}



### $(k_1,k_2,k_3)=(1,2,4)$

In [90]:
graphname="G6M2"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","2")], multiedges=True)

data=unpickle(graphname) #Size=120


dic_l=analyse_graphname(G,data)

Start= 0 ---- 0 ----terminator: 1
Start= 0 ---- 1 ----terminator: 1


In [ ]:
#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l,f=True)

### $(k_1,k_2,k_3)=(1,3,3)$

In [25]:
graphname="G6M3"  
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","3")], multiedges=True)

data=unpickle(graphname) #Size=120

dic_l=analyse_graphname(G,data)

Start= 0 ---- 0 ----terminator: False


KeyboardInterrupt: 

In [ ]:
#Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l,f=True)

### $(k_1,k_2,k_3)=(1,2,5)$

### $(k_1,k_2,k_3)=(1,3,4)$

## $\Gamma:=\Gamma_{k_1 k_2 k_3}$ for $k_1\ge 2$  <a name="s24"></a>

We now consider the dimension of $P_{ST}^{\sigma_{\Gamma}}$ for $ \sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ for graphs   $\Gamma:=\Gamma_{k_1 k_2 k_3}$ with  $(k_1,k_2,k_3)=\{(2,2,2),(2,2,3),(2,2,4),(2,3,3)\}$. 

### $(k_1,k_2,k_3)=(2,2,2)$



### $(k_1,k_2,k_3)=(2,2,3)$



### $(k_1,k_2,k_3)=(2,2,4)$



### $(k_1,k_2,k_3)=(2,3,3)$

## $\Gamma \in \{ G_1,G_2,G_3,F\}$ <a name="s25"></a>

We now consider the dimension of $P_{ST}^{\sigma_{\Gamma}}$ for $ \sigma_{\Gamma} \in \Sigma_{\Gamma,T,D_{T}}^{d}$ for graphs   $\Gamma\in \{G_1,G_2,G_3,F\}$.

### $F$

In [26]:
graphname="FV_G6M3M14"  
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","3"),("1","4")], multiedges=True)

data=unpickle(graphname) #Size=504

# dic_l=analyse_graphname(G,data,ind_breaks=0)


In [29]:
dic_l=analyse_graphname_FV(5,graphname,G,data,ind_breaks=0)

Start= 0 ---- 0 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 1 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 2 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 3 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 4 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 5 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 6 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 7 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 8 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 9 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 10 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 11 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 12 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 13 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 14 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 15 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 16 ----terminator: False
dim_tree_poly: 5
Start= 0 ---- 17 ----terminator: False
dim_tree_poly: 5
St

### $G_1$

In [26]:
graphname="G4M02M13"
G=Graph([("0","1"),("1","2"),("2","3"),("3","0"),("0","2"),("1","3")], multiedges=True)

data=unpickle(graphname) #Size=10

dic_l=analyse_graphname(G,data,ind_breaks=0)

Start= 0 ---- 0 ----terminator: False
Start= 0 ---- 1 ----terminator: False
Start= 0 ---- 2 ----terminator: False
Start= 0 ---- 3 ----terminator: False
Start= 0 ---- 4 ----terminator: False


KeyboardInterrupt: 

In [93]:
# #Store list of dictionaries
# pickle_dic_l(graphname,dic_l)

In [94]:
analyse_table(dic_l,f=True)

Total number of wsc 10
Number of wsc that do/do not satisfy graph poly == tree poly: {True: 10}
Number of polytopes of a given (dimension:number of at dim): {3: 10}
Number of wsc that are/are not generated by phi_rep: {True: 10}
Number of wsc that are/are not generated by phi_av: {True: 10}
Number of wsc that are/are not generated by phi_can: {True: 1, False: 9}
Number of wsc that are/are not generated by phi_av_translated1: {True: 10}
Number of wsc that are/are not generated by phi_av_translated2: {True: 10}



### $G_2$

In [ ]:
graphname="G5M02M03"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2"),("0","3")], multiedges=True)

data=unpickle(graphname) #Size=24

dic_l=analyse_graphname(G,data,ind_breaks=0)

In [ ]:
#Store list of dictionaries
pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l,f=True)

### $G_3$

In [ ]:
graphname="G5M02M03M14"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2"),("0","3"),("1","4")], multiedges=True)

data=unpickle(graphname) #Size=82

dic_l=analyse_graphname(G,data,ind_breaks=0)

In [ ]:
#Store list of dictionaries
pickle_dic_l(graphname,dic_l)

In [ ]:
analyse_table(dic_l,f=True)